In [47]:
import gensim
word2vecmodel = gensim.models.KeyedVectors.load_word2vec_format(path.expanduser('~/work/jupyter_notebooks/notebooks/GoogleNews-vectors-negative300.bin'), binary=True)

In [49]:
word2vecmodel.save(path.expanduser('~/work/data/word2vec.bin'))

In [52]:
word2vecmodel = gensim.models.KeyedVectors.load(path.expanduser('~/work/data/word2vec.bin'), mmap='r')

In [43]:
import requests
url = 'https://www.ted.com/talks/craig_costello_cryptographers_quantum_computers_and_the_war_for_information'
title = url.split('/')[-1]
metaURL = 'https://api.ted.com/v1/talks/{}.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
r = requests.get(url = metaURL.format(title))
data = r.json()
id = data['talk']['id']
# [tag['tag'] for tag in data['talk']['tags']]

50956

In [10]:
import numpy as np
import pickle
import pandas as pd
from os import path
import re

with open(path.expanduser('~/work/data/data.pkl'), "rb") as input_file:
    data = pickle.load(input_file)
    
tags = [re.sub('\'','',taglist)[1:-1].split(', ') for taglist in data['tags'].values]
file = open(path.expanduser('~/work/data/tags.txt'), "w")
for talktags in tags:
    file.write(' '.join(talktags) + '\n')
file.close()

In [36]:
import requests
import numpy as np
import nltk
import re
import gensim
nltk.download('stopwords')
nltk.download('wordnet')

def retrieve_prepare_subtitles(url):
    
    metaDataURL = 'https://api.ted.com/v1/talks/{}.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
    subURL = 'https://api.ted.com/v1/talks/{}/subtitles.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
    r = requests.get(url = subURL.format(id))
    data = r.json() 
    transcript = []
    for dict in np.arange(len(data)-1):
        text = data[str(dict)]['caption']['content']
        text = re.sub(r'\((.*?)\)', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
        text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
        text = [nltk.corpus.wordnet.morphy(w) if nltk.corpus.wordnet.morphy(w) else w for w in text]
        transcript.extend(text)
    return ' '.join(transcript)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
import os

file = open(path.expanduser('~/work/test/lftm/doc.txt'), "w")
file.write(retrieve_prepare_subtitles(176)) 
file.close()

model = 'LFLDAinf'
paras = 'TEDLFLDA.paras'
corpus = 'doc.txt'
initer = '500'
niter = '50'
topn = '20'
name = 'TEDLFLDAinf'
sstep = '0'

os.chdir(os.path.expanduser('~/work/test/lftm/'))
os.system('java -jar jar/LFTM.jar -model {} -paras {} -corpus {} -initers {} -niters {} -twords {} -name {} -sstep {}'.format(
    model,
    paras,
    corpus,
    initer,
    niter,
    topn,
    name,
    sstep))

0

In [56]:
file = open(path.expanduser('~/work/test/lftm/TEDLFLDAinf.theta'), "r")
topics = file.readline()
file.close()

In [ ]:
def train(datapath):
    text = []
    with open(datapath,"r+") as f:
        while True:
            vec = f.readline()
            text.append(text)

In [2]:
import pickle
from os import path
import pandas as pd

with open(path.expanduser('~/work/data/data.pkl'), "rb") as input_file:
    data = pickle.load(input_file)

In [ ]:
import json
import requests
import pandas as pd
import numpy as np

metaDataURL = 'https://api.ted.com/v1/talks/{}.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
subURL = 'https://api.ted.com/v1/talks/{}/subtitles.json?api-key=uzdyad5pnc2mv2dd8r8vd65c'
_id=1

talk_meta = pd.DataFrame()
talk_sub = pd.DataFrame()
talk = {}
sub = {}
while True:
    response_meta = requests.get(metaDataURL.format(_id))
    response_meta = response_meta.json()
    if 'error' in response_meta:
        print("Id",_id, "not found.")
        _id += 1
        continue
    response_sub = requests.get(subURL.format(_id))
    response_sub = response_sub.json()
    if 'error' in response_sub:
        print("Sub for",_id, "not found.")
        _id += 1
        continue
    talk['talk_id'] = _id
    talk['name'] = response_meta['talk']['name']
    talk['tags'] = ','.join([tag['tag'] for tag in response_meta['talk']['tags']])
    talk_meta = talk_meta.append(talk, ignore_index=True)
    
    chapter_id = 0
    chapter = []
    
    no_caption = len(response_sub)
    chapter.append(response_sub['0']['caption']['content'])
    
    for dict in np.arange(2,no_caption - 2):
        if response_sub[str(dict)]['caption']['startOfParagraph']:
            sub['talk_id'] = _id
            sub['id'] = chapter_id
            sub['transcript'] = ' '.join(chapter)
            talk_sub = talk_sub.append(sub, ignore_index=True)
            chapter_id += 1
            chapter = []
            chapter.append(response_sub[str(dict)]['caption']['content'])
        else:
            chapter.append(response_sub[str(dict)]['caption']['content'])
            
    if response_sub[str(no_caption - 2)]['caption']['startOfParagraph']:
        sub['talk_id'] = _id
        sub['id'] = chapter_id
        sub['transcript'] = ' '.join(chapter)
        talk_sub = talk_sub.append(sub, ignore_index=True)
        chapter_id += 1
        chapter = []
        chapter.append(response_sub[str(no_caption - 2)]['caption']['content'])
        sub['talk_id'] = _id
        sub['id'] = chapter_id
        sub['transcript'] = ' '.join(chapter)
        talk_sub = talk_sub.append(sub, ignore_index=True)
    else:
        sub['talk_id'] = _id
        sub['id'] = chapter_id
        chapter.append(response_sub[str(no_caption - 2)]['caption']['content'])
        sub['transcript'] = ' '.join(chapter)
        talk_sub = talk_sub.append(sub, ignore_index=True)
                
#         text = re.sub(r'\((.*?)\)', ' ', text)
#         text = re.sub(r'\d+', '', text)
#         text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
#         text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
#         text = [nltk.corpus.wordnet.morphy(w) if nltk.corpus.wordnet.morphy(w) else w for w in text]
#         transcript.extend(text)
    print("Loaded talk",_id)
    _id += 1

In [84]:
talk_sub.groupby('talk_id').count()

,id,transcript
talk_id,,
1.0,65,65
2.0,39,39
3.0,22,22
4.0,28,28
5.0,28,28
...,...,...
3642.0,12,12
3671.0,8,8
3680.0,35,35


In [93]:
talk_sub[(talk_sub.talk_id ==1) & (talk_sub.id == 0)].sort_values('id').values

array([[0.0, 1.0,
        "Thank you so much, Chris. to come to this stage twice;\nI'm extremely grateful. I have been blown away by this conference, and I want to thank all of you\nfor the many nice comments about what I had to say the other night. And I say that sincerely, partly because (Mock sob) I need that."]],
      dtype=object)

In [94]:
talk_meta

,name,tags,talk_id
0,Al Gore: Averting the climate crisis,"alternative energy,cars,climate change,culture...",1.0
1,Amy Smith: Simple designs to save a life,"MacArthur grant,alternative energy,design,engi...",2.0
2,Ashraf Ghani: How to rebuild a broken state,"business,corruption,culture,economics,entrepre...",3.0
3,Burt Rutan: The real future of space exploration,"NASA,aircraft,business,design,engineering,entr...",4.0
4,Chris Bangle: Great cars are great art,"art,business,cars,design,industrial design,inv...",5.0
...,...,...,...
2555,Niti Bhan: The hidden opportunities of the inf...,"Africa,business,creativity,culture,economics,i...",3642.0
2556,Matilda Ho: The future of good food in China,"TED Fellows,agriculture,china,entrepreneur,far...",3671.0
2557,Gretchen Carlson: How we can end sexual harass...,"activism,journalism,law,leadership,personal gr...",3680.0
2558,Dan Gartenberg: The brain benefits of deep sle...,"Human body,TED Residency,brain,health,memory,s...",3686.0


In [219]:
talk_data = talk_meta.set_index('talk_id').join(talk_sub.set_index('talk_id'), 'talk_id')

In [220]:
talk_data = talk_meta.set_index('talk_id').join(talk_sub.set_index('talk_id'), on = 'talk_id')
talk_meta = talk_meta[~talk_meta['talk_id'].isin(talk_data[talk_data['transcript'].isna()].index)]

In [95]:
with open(path.expanduser('~/work/data/talk_meta.pkl'), 'wb') as output:
    pickle.dump(talk_meta, output, pickle.HIGHEST_PROTOCOL)
with open(path.expanduser('~/work/data/talk_sub.pkl'), 'wb') as output:
    pickle.dump(talk_sub, output, pickle.HIGHEST_PROTOCOL)

In [1]:
from os import path
import pickle

with open(path.expanduser('~/work/data/talk_meta.pkl'), "rb") as input_file:
    talk_meta = pickle.load(input_file)
with open(path.expanduser('~/work/data/talk_sub.pkl'), "rb") as input_file:
    talk_sub = pickle.load(input_file)

In [6]:
talk_sub.groupby('talk_id').count().sort_values('transcript')

,id,transcript
talk_id,,
1263.0,1,1
1430.0,1,1
1677.0,1,1
2147.0,1,1
2273.0,2,2
...,...,...
2010.0,215,215
254.0,220,220
2209.0,230,230


In [4]:
talk_meta

,name,tags,talk_id
0,Al Gore: Averting the climate crisis,"alternative energy,cars,climate change,culture...",1.0
1,Amy Smith: Simple designs to save a life,"MacArthur grant,alternative energy,design,engi...",2.0
2,Ashraf Ghani: How to rebuild a broken state,"business,corruption,culture,economics,entrepre...",3.0
3,Burt Rutan: The real future of space exploration,"NASA,aircraft,business,design,engineering,entr...",4.0
4,Chris Bangle: Great cars are great art,"art,business,cars,design,industrial design,inv...",5.0
...,...,...,...
2573,Anjan Sundaram: Why I risked my life to expose...,"Africa,TED Fellows,corruption,journalism,war",3363.0
2574,Mike Kinney: A pro wrestler's guide to confidence,"business,creativity,humor,identity",3364.0
2575,Christen Reighter: I don't want children -- st...,"TEDx,children,family,identity,parenting,social...",3365.0
2576,"Chris Sheldrick: A precise, three-word address...","communication,cooperation,global development,i...",3366.0


In [2]:
!pip install nltk

     |████████████████████████████████| 1.5MB 1.6MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449906 sha256=28275f2290bf7b36a75865276ea58317c4c97f28a7560fe571a6fce08335cd4b
  Stored in directory: /home/jovyan/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
def prepare_text(text):
    text = re.sub(r'\((.*?)\)', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text.lower())
    text = [w for w in text if not((w in ['thank', 'much']) or (w in nltk.corpus.stopwords.words('english')))]
    text = [nltk.corpus.wordnet.morphy(w) if nltk.corpus.wordnet.morphy(w) else w for w in text]
    return text

In [34]:
talk_sub[talk_sub.talk_id == 1]['transcript'].apply(lambda x: prepare_text(x)).values

array([list(['chris', 'truly', 'great', 'honor', 'opportunity', 'come', 'stage', 'twice', 'extremely', 'grateful', 'blow', 'away', 'conference', 'want', 'many', 'nice', 'comment', 'say', 'night', 'say', 'sincerely', 'partly', 'need']),
       list([]), list(['put', 'position']), list([]),
       list(['fly', 'air', 'force', 'two', 'eight', 'years']), list([]),
       list(['take', 'shoes', 'boot', 'get', 'airplane']), list([]),
       list([]),
       list(['tell', 'one', 'quick', 'story', 'illustrate', 'like']),
       list([]), list(['true', 'story', 'every', 'bit', 'true']),
       list(['soon', 'tipper', 'left', 'white', 'house']), list([]),
       list(['driving', 'home', 'nashville', 'little', 'farm', 'mile', 'east', 'nashville', 'driving']),
       list([]), list(['know', 'sound', 'like', 'little', 'thing']),
       list([]),
       list(['look', 'rear', 'view', 'mirror', 'sudden', 'hit', 'motorcade', 'back']),
       list([]), list(['hear', 'phantom', 'limb', 'pain']), list([])

In [10]:
tag2idx = {}
i=0
talktags = [taglist.split(',') for taglist in talk_meta['tags'].values]
for taglist in talktags:
    for tag in taglist:
        if tag not in tag2idx and not 'ted' in tag.lower():
            tag2idx[tag] = i
            i += 1
tag2idx

{'alternative energy': 0,
 'cars': 1,
 'climate change': 2,
 'culture': 3,
 'environment': 4,
 'global issues': 5,
 'science': 6,
 'sustainability': 7,
 'technology': 8,
 'MacArthur grant': 9,
 'design': 10,
 'engineering': 11,
 'industrial design': 12,
 'invention': 13,
 'simplicity': 14,
 'business': 15,
 'corruption': 16,
 'economics': 17,
 'entrepreneur': 18,
 'global development': 19,
 'investment': 20,
 'military': 21,
 'policy': 22,
 'politics': 23,
 'poverty': 24,
 'NASA': 25,
 'aircraft': 26,
 'flight': 27,
 'rocket science': 28,
 'art': 29,
 'transportation': 30,
 'DNA': 31,
 'biodiversity': 32,
 'biology': 33,
 'biotech': 34,
 'ecology': 35,
 'genetics': 36,
 'oceans': 37,
 'computers': 38,
 'entertainment': 39,
 'interface design': 40,
 'media': 41,
 'music': 42,
 'performance': 43,
 'software': 44,
 'New York': 45,
 'architecture': 46,
 'cities': 47,
 'collaboration': 48,
 'death': 49,
 'disaster relief': 50,
 'interview': 51,
 'memory': 52,
 'urban planning': 53,
 'innova

In [9]:
len(tag2idx)

423

In [1]:
from os import path
import pickle

with open(path.expanduser('~/work/data/talk_pre.pkl'), "rb") as input_file:
    talk_pre = pickle.load(input_file)

In [3]:
# !sudo apt update -y
!pip install nltk
!pip install gensim

     |████████████████████████████████| 24.2MB 138kB/s eta 0:00:01    |█████▋                          | 4.2MB 165kB/s eta 0:02:01     |███████▎                        | 5.5MB 140kB/s eta 0:02:14
     |████████████████████████████████| 71kB 156kB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 128kB/s eta 0:00:01
     |████████████████████████████████| 133kB 152kB/s eta 0:00:01
     |████████████████████████████████| 71kB 190kB/s eta 0:00:01
     |████████████████████████████████| 5.9MB 159kB/s eta 0:00:01     |████████████████████▏           | 3.7MB 116kB/s eta 0:00:19
     |████████████████████████████████| 552kB 153kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=b01c5908cf40f0465b1ebb56f8b0fedcd33881387e03c30b53470dd8c6601298
  Stored in directory: /home/jovyan/.cache/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


In [8]:
import re
import pandas as pd
from os import path
import nltk
import gensim
import time
from IPython.display import display, clear_output
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pasquale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pasquale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

In [5]:
data.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [4]:
data.groupby('target_names').count()

,content,target
target_names,,
alt.atheism,480,480
comp.graphics,584,584
comp.os.ms-windows.misc,591,591
comp.sys.ibm.pc.hardware,590,590
comp.sys.mac.hardware,578,578
comp.windows.x,593,593
misc.forsale,585,585
rec.autos,594,594
rec.motorcycles,598,598


In [9]:
lem = nltk.stem.WordNetLemmatizer()
def prepare_text(text):
    text = text.lower()
    text = re.sub('from:.+\n', '', text)
    text = re.sub('article-i.d.:.+\n', '', text)
    text = re.sub('nntp-posting-host:.+\n', '', text)
    text = re.sub('organization:.+\n', '', text)
    text = re.sub('x-newsreader:.+\n', '', text)
    text = re.sub('distribution:.+\n', '', text)
    text = re.sub('reply-to:.+\n', '', text)
    text = re.sub('<.+>', '', text)
    text = re.sub(r'\d+', '', text)
    text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text = [lem.lemmatize(w) for w in text]
    text = [w for w in text if not(w in ['from', 'subject', 'edu', 'use', 'lines', 'ke'] or w in nltk.corpus.stopwords.words('english')) and len(w)>=3 and '__' not in w]
    text = [w for w in text if len(w)>=3]
    return text

In [10]:
data['content'] = data.content.apply(lambda x: prepare_text(x))

KeyboardInterrupt: 

In [ ]:
with open('20ng.txt', 'w') as f:
     for d in data['content']:
            f.write(' '.join(d))
            f.write('\n')

In [ ]:
data_words = data['content'].values

id2word = gensim.corpora.Dictionary(data_words)
id2word.filter_n_most_frequent(10)

# corpus = [id2word.doc2bow(text) for text in data_words]

In [ ]:
def filter_text(text, tokens):
    filtered_text = []
    for word in text:
            filtered_text.append(word)
    return filtered_text

In [ ]:
data['content'] = data.content.apply(lambda x: filter_text(x, id2word.token2id))

In [ ]:
data['content'] = data.content.apply(lambda x: ' '.join(x))

In [ ]:
f = open(path.expanduser('~/work/data/20NG.txt'),"w+") 
for text in data['content']:
    f.write(text+'\n')
f.close()

In [6]:
ref = pd.read_csv('~/work/data/wikipedia_utf8_filtered_20pageviews.csv',header=None)

In [7]:
ref.head()

,0,1
0,wikipedia-23885690,Research Design and Standards Organization T...
1,wikipedia-23885928,The Death of Bunny Munro The Death of Bunny ...
2,wikipedia-23886057,Management of prostate cancer Treatment for ...
3,wikipedia-23886425,Cheetah reintroduction in India Reintroducti...
4,wikipedia-23886491,Langtang National Park The Langtang National...


In [3]:
lem = nltk.stem.WordNetLemmatizer()

def simple_prepare_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text = [lem.lemmatize(w) for w in text]
    text = [w for w in text if len(w)>=3]
    text = ' '.join(text)
    return text

In [24]:
# ref[1] = ref[1].apply(lambda x: simple_prepare_text(x))

In [28]:
len(ref)

463819

In [29]:
f = open(path.expanduser('~/work/data/wiki.txt'),"w+") 
i = 0
total = len(ref)
for text in ref[1]:
    clear_output(wait=True)
    start = time.time()
    text = simple_prepare_text(text)
    f.write(text+'\n')
    dur = time.time() - start
    display('Progress '+ str(100*i/total) + ' % - Duration per doc: '+ str(dur*1000) + ' ms')
    i+=1
f.close()

'Progress 99.9997843986555 % - Duration per doc: 4.185199737548828 ms'

In [100]:
print(str(5*463819/1000/60) + ' mins')

38.65158333333333 mins


In [37]:
f = open(path.expanduser('~/work/data/sw.txt'),"w+") 

for w in simple_prepare_text(' '.join(nltk.corpus.stopwords.words('english'))).split():
#     s|\<the\>||g
    f.write('s|\<'+w+'\>||g\n')
f.close()

In [5]:
import pandas as pd
import numpy as np
import pickle
from os import path
import re
import nltk
import gensim
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
lem = nltk.stem.WordNetLemmatizer()
def prepare_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text = [lem.lemmatize(w) for w in text]
    text = [w for w in text if w not in nltk.corpus.stopwords.words('english')]
    text = [w for w in text if len(w)>=3]
    return text

In [ ]:
f1 = open(path.expanduser('~/work/data/asrael.txt'),"r") 
f2 = open(path.expanduser('~/work/data/asrael_cleaned.txt'),"w") 

i = 0
total = 125516
for line in f1:
    clear_output(wait=True)
    start = time.time()
    text = ' '.join(prepare_text(line))
    f2.write(text+'\n')
    dur = time.time() - start
    display('Progress '+ str(100*i/total) + ' % - Duration per doc: '+ str(dur*1000) + ' ms')
    i+=1
f1.close()
f2.close()